In [1]:
import numpy as np
import pandas as pd
import os
#input=pd.read_excel('/content/drive/MyDrive/Blackcoffer/20211030 Test Assignment/Input.xlsx')
input=pd.read_excel('/content/drive/MyDrive/Blackcoffer/20211030 Test Assignment/Input.xlsx')
input.set_index('URL_ID', inplace=True)
input


,URL
URL_ID,
123.0,https://insights.blackcoffer.com/rise-of-telem...
321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
4321.0,https://insights.blackcoffer.com/rise-of-telem...
432.0,https://insights.blackcoffer.com/rise-of-telem...
...,...
50921.0,https://insights.blackcoffer.com/coronavirus-i...
51382.8,https://insights.blackcoffer.com/coronavirus-i...
51844.6,https://insights.blackcoffer.com/what-are-the-...


In [2]:
import requests
from bs4 import BeautifulSoup
import tqdm as tqdm
folder_path = '/content/drive/MyDrive/Blackcoffer/text_folder/'
for index, row in input.iterrows():
    url = str(row['URL'])
    file = folder_path+str(index)+'.txt'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        text_content = [p.get_text() for p in paragraphs]
        text = '\n'.join(text_content)
        with open(file, 'w+') as file:
            file.write(text)
    else:
        print("Error accessing the URL:", response.status_code)
        print(url)
        print(index)
        text="Error accessing the URL: "+str(response.status_code)
        with open(file, 'w+') as file:
          file.write(text)
          file.seek(0)
          f=file.read()
          print(f)


Error accessing the URL: 404
https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
11668.0
Error accessing the URL: 404
Error accessing the URL: 404
https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
17671.4
Error accessing the URL: 404


In [3]:
pip install nltk

In [4]:

def create_stop_words(stop_words_path):
  stop_words=[]
  for root, dirs, files in os.walk(stop_words_path):
    for file in files:
        file = os.path.join(root, file)
        with open(file,'r',encoding='latin-1') as f:
          lines=f.readlines()
        for line in lines:
          word=line.strip().split('|')[0].strip()
          stop_words.append(word.lower())
  return stop_words




def create_word_dictionary(master_dictionary_path_positive,master_dictionary_path_negative,stop_words):
    word_dictionary = {'positive': [], 'negative': []}
    with open(master_dictionary_path_positive, 'r',encoding='latin-1') as file:
        lines = file.readlines()
    for line in lines:
        word = line.strip()
        if word.lower() not in stop_words:
            word_dictionary['positive'].append(word.lower())
    with open(master_dictionary_path_negative, 'r',encoding='latin-1') as file:
        lines = file.readlines()
    for line in lines:
        word = line.strip()
        if word.lower() not in stop_words:
            word_dictionary['negative'].append(word.lower())


    return word_dictionary



stop_words_path = '/content/drive/MyDrive/Blackcoffer/20211030 Test Assignment/StopWords'
stop_words = create_stop_words(stop_words_path)

master_dictionary_path_positive = '/content/drive/MyDrive/Blackcoffer/20211030 Test Assignment/MasterDictionary/positive-words.txt'
master_dictionary_path_negative = '/content/drive/MyDrive/Blackcoffer/20211030 Test Assignment/MasterDictionary/negative-words.txt'
word_dictionary=create_word_dictionary(master_dictionary_path_positive,master_dictionary_path_negative,stop_words)



In [5]:
output=pd.read_excel('/content/drive/MyDrive/Blackcoffer/20211030 Test Assignment/Output Data Structure.xlsx')
output.set_index('URL_ID', inplace=True)
output.columns

Index(['URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'],
      dtype='object')

In [6]:
import nltk
nltk.download('punkt')
nltk.download('cmudict')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


True

In [7]:
import re
from nltk.corpus import cmudict
from nltk.tokenize import word_tokenize,sent_tokenize

def tokenize_text_file(file_path):
    tokens=[]
    sentences=[]
    with open(file_path, 'r') as file:
        text = file.read()
    tokens=word_tokenize(text)
    tokens = [x.lower() for x in tokens if x.lower() not in stop_words]
    sentences=sent_tokenize(text)

    return tokens,sentences

def positive_score(tokens):
  c=0
  for token in tokens:
    if token in word_dictionary['positive']:
      c=c+1
  return c

def negative_score(tokens):
  c=0
  for token in tokens:
    if token in word_dictionary['negative']:
      c=c+1
  return c

def polarity_score(positive_score,negative_score):
   return (positive_score-negative_score)/ ((positive_score+negative_score)+0.000001)

def subjectivity_score(positive_score,negative_score,tokens):
   return (positive_score+negative_score)/ ((len(tokens)) + 0.000001)

def avg_sent_length(n,m):
   return n/m



cmud = cmudict.dict()

def count_syllables(word):
    word = re.sub('[^a-zA-Z]', '', word.lower())
    if word in cmud:
        return [len(list(y for y in x if y[-1].isdigit())) for x in cmud[word]][0]
    else:
        vowels = 'aeiou'
        return len([c for c in word if c in vowels])

def count_complex(tokens, syllable_threshold):
    multi_syllable_words = []
    for word in tokens:
        if count_syllables(word) > syllable_threshold:
            multi_syllable_words.append(word)

    return len(multi_syllable_words)



import re

def count_syllables(token):
    word = re.sub('[^a-zA-Z]', '', token.lower())
    if word in cmud:
        return [len(list(y for y in x if y[-1].isdigit())) for x in cmud[word]][0]
    else:
        vowels = 'aeiou'
        count = 0
        prev_char = None

        for char in word:
            if char in vowels and prev_char is not None and prev_char not in vowels:
                count += 1
            prev_char = char

        return count


def count_syllables_per_word(tokens):
    syllable_counts = [count_syllables(token) for token in tokens]
    return syllable_counts



def count_personal_pronouns(file):
    with open(file, 'r') as file:
        text = file.read()
    pronouns = ["I", "we", "my", "ours", "us"]
    pattern = r"\b(?:{})\b".format("|".join(pronouns))
    exclude_pattern = r"\bUS\b"
    matches = re.findall(pattern, text, flags=re.IGNORECASE)
    exclude_matches = re.findall(exclude_pattern, text, flags=re.IGNORECASE)
    matches = [match for match in matches if match not in exclude_matches]
    return len(matches)

def average_word_length(tokens):
    total_characters = sum(len(token) for token in tokens)
    total_words = len(tokens)

    if total_words == 0:
        return 0

    average_word_length = total_characters / total_words
    return average_word_length






In [8]:
filepath='/content/drive/MyDrive/Blackcoffer/text_folder'
out=pd.DataFrame(index=output.index,columns=output.columns)
for index, row in output.iterrows():
    url = str(row['URL'])
    tokens,sentences=tokenize_text_file(filepath+'/'+str(index)+'.txt')
    row['POSITIVE SCORE'] = positive_score(tokens)
    row['NEGATIVE SCORE'] = negative_score(tokens)
    row['POLARITY SCORE'] = polarity_score( row['POSITIVE SCORE'],row['NEGATIVE SCORE'])
    row['SUBJECTIVITY SCORE'] = subjectivity_score(row['POSITIVE SCORE'],row['NEGATIVE SCORE'],tokens)
    row['AVG SENTENCE LENGTH'] = avg_sent_length(len(tokens),len(sentences))
    row['PERCENTAGE OF COMPLEX WORDS'] = count_complex(tokens,2)/len(tokens)
    row['FOG INDEX'] =  0.4 * (row['AVG SENTENCE LENGTH'] +  row['PERCENTAGE OF COMPLEX WORDS'] )
    row['AVG NUMBER OF WORDS PER SENTENCE']= row['AVG SENTENCE LENGTH']
    row['COMPLEX WORD COUNT'] = count_complex(tokens,2)
    row['WORD COUNT'] = len(tokens)
    row['SYLLABLE PER WORD'] = count_syllables_per_word(tokens)
    row['PERSONAL PRONOUNS'] = count_personal_pronouns(filepath+'/'+str(index)+'.txt')
    row['AVG WORD LENGTH'] = average_word_length(tokens)
    out = out.append(row, ignore_index=True)


<ipython-input-8-29953db5e80c>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(row, ignore_index=True)


In [9]:
out.shape

(228, 14)

In [10]:
out.isna().sum()

URL                                 114
POSITIVE SCORE                      114
NEGATIVE SCORE                      114
POLARITY SCORE                      114
SUBJECTIVITY SCORE                  114
AVG SENTENCE LENGTH                 114
PERCENTAGE OF COMPLEX WORDS         114
FOG INDEX                           114
AVG NUMBER OF WORDS PER SENTENCE    114
COMPLEX WORD COUNT                  114
WORD COUNT                          114
SYLLABLE PER WORD                   114
PERSONAL PRONOUNS                   114
AVG WORD LENGTH                     114
dtype: int64

In [11]:
out=out.tail(228-114)
out.index=output.index
out

,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
URL_ID,,,,,,,,,,,,,,
123.0,https://insights.blackcoffer.com/rise-of-telem...,84,24,0.555556,0.091993,13.976190,0.336457,5.725059,13.976190,395,1174,"[3, 2, 3, 2, 3, 3, 2, 3, 1, 1, 2, 3, 0, 0, 0, ...",3,6.467632
321.0,https://insights.blackcoffer.com/rise-of-e-hea...,43,14,0.508772,0.115619,17.000000,0.356998,6.942799,17.000000,176,493,"[3, 2, 3, 2, 3, 3, 2, 3, 1, 1, 2, 3, 0, 0, 0, ...",4,6.501014
2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,23,28,-0.098039,0.063041,11.082192,0.330037,4.564892,11.082192,267,809,"[3, 2, 3, 2, 3, 3, 2, 3, 1, 1, 2, 3, 0, 0, 0, ...",4,6.000000
4321.0,https://insights.blackcoffer.com/rise-of-telem...,39,27,0.181818,0.071274,14.468750,0.353132,5.928753,14.468750,327,926,"[3, 2, 3, 2, 3, 3, 2, 3, 1, 1, 2, 3, 0, 0, 0, ...",8,6.209503
432.0,https://insights.blackcoffer.com/rise-of-telem...,39,27,0.181818,0.071274,14.468750,0.353132,5.928753,14.468750,327,926,"[3, 2, 3, 2, 3, 3, 2, 3, 1, 1, 2, 3, 0, 0, 0, ...",8,6.209503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50921.0,https://insights.blackcoffer.com/coronavirus-i...,10,8,0.111111,0.039735,17.423077,0.280353,7.081372,17.423077,127,453,"[3, 2, 3, 2, 3, 3, 2, 3, 1, 1, 2, 3, 0, 0, 0, ...",2,5.615894
51382.8,https://insights.blackcoffer.com/coronavirus-i...,26,62,-0.409091,0.076256,21.370370,0.258232,8.651441,21.370370,298,1154,"[3, 2, 3, 2, 3, 3, 2, 3, 1, 1, 2, 3, 0, 0, 0, ...",4,5.679376
51844.6,https://insights.blackcoffer.com/what-are-the-...,86,32,0.457627,0.097119,16.418919,0.311934,6.692341,16.418919,379,1215,"[3, 2, 3, 2, 3, 3, 2, 3, 1, 1, 2, 3, 0, 0, 0, ...",1,5.986008


In [12]:
out.to_csv('/content/drive/MyDrive/Blackcoffer/output.csv')

In [13]:
from google.colab import files
files.download('/content/drive/MyDrive/Blackcoffer/output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>